### iRage IOC Assignment - 27th Feb 2022

### Assumptions
- For a trade to be IOC, by definition - it has to have the same timestamp for N/M and T Type trade as it is immediate or cancel order
- For any IOC Trade, there is exactly one N or M order and also, both N and M orders cannot occur at the same timestamp
- Trade and Cancel order also cannot happen at the same timestamp 
- Same ExchId has to be present in T and N/M order for IOC Trade
- Have taken absolute values for Bought and Sold Straddles

#### Importing Modules

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

#### Reading Data

In [2]:
df1 = pd.read_csv('dataset_instrument1.csv')
df2 = pd.read_csv('dataset_instrument2.csv')

#### Finding IOC Trades

In [3]:
def find_ioc_trades(df):
    
    # Drop trades/exchstamp where only 1 trade is present as for the trade to be IOC, it has to have min 2 trades - 1 for N/M and 1 for T
    df_non_unique = df[df.ExchTstamp.duplicated(keep=False)]
    # Storing value as price*qty for T type trades
    df_non_unique['value'] = df_non_unique.apply(lambda row: row['price']*row['qty'] if row['Type'] == 'T' else 0, axis=1)
    
    # Grouping the trades by ExchTstamp to identify possible IOC trades
    gb = df_non_unique.groupby(by=['ExchTstamp'])    
    df_groups = [gb.get_group(x) for x in gb.groups]

    # Initiating count and value of IOC trades
    ioc_trades_cnt = 0
    ioc_trades_value = 0
    ioc_df_list = []
    
    # Looping over all groups
    for df in df_groups:
        # Condition to check that the group has to have either N/M and T to classify as IOC Trade
        if (('N' in df.Type.values) | ('M' in df.Type.values)) & ('T' in df.Type.values) & ('X' not in df.Type.values):
            # Adding those numbers and values to the respective variables
            ioc_df_list.append(df)
            ioc_trades_cnt += len(df[df['Type'] == 'T'])
            ioc_trades_value += df['value'].sum()
            
    return [ioc_trades_cnt, ioc_trades_value, ioc_df_list]

In [4]:
%%time
# Calling function to find out IOC trades for each Instruments - Call and Put
[call_ioc_trades_cnt, call_ioc_trades_value, call_ioc_df_list] = find_ioc_trades(df1)
[put_ioc_trades_cnt, put_ioc_trades_value, put_ioc_df_list] = find_ioc_trades(df2)

print('CALL INSTRUMENT :')
print('Total No. of IOC Traded - {}'.format(call_ioc_trades_cnt))
print('Total Value of IOC Traded - {:,}'.format(call_ioc_trades_value))
print('---------------------------------------------------------')
print('PUT INSTRUMENT :')
print('Total No. of IOC Traded - {}'.format(put_ioc_trades_cnt))
print('Total Value of IOC Traded - {:,}'.format(put_ioc_trades_value))

CALL INSTRUMENT :
Total No. of IOC Traded - 6546
Total Value of IOC Traded - 11,868,147,750
---------------------------------------------------------
PUT INSTRUMENT :
Total No. of IOC Traded - 10539
Total Value of IOC Traded - 11,094,982,500
Wall time: 6.02 s


### Finding Straddles from IOC Trades

In [5]:
# Concatenating all Put Instrument IOC trades in a df
put_ioc_df = pd.concat(put_ioc_df_list)
# Filtering it out by T Type trades
put_ioc_df = put_ioc_df[put_ioc_df['Type']=='T'] 

dict_delta = {'nanoseconds':1,'microseconds':10**3,'milliseconds':10**6,'second':10**9}

def find_straddles(deltaval):
    
    # Finding out Delta based on the passed input - 100 nanoseconds or 10 microseconds, etc.
    if(deltaval.split()[1]=='second' or deltaval.split()[1]=='seconds'):
        delta = 10**9 * float(deltaval.split()[0])
    else:
        delta = [val for key, val in dict_delta.items() if deltaval.split()[1] in key][0] * float(deltaval.split()[0])
    
    # Initiating counts and values
    cnt = 0
    bought_straddles_cnt = 0
    sold_straddles_cnt = 0
    bought_straddles_value = 0
    sold_straddles_value = 0

    # Looping over each Call IOC Trade df
    for df_call in call_ioc_df_list:
        # Sorting df by Type
        df_call = df_call.sort_values(by=['Type'])
        
        # Extracting out the ExchTstamp for the IOC Trade from the CALL Instrument
        timestamp = df_call.ExchTstamp.iloc[0]
        
        # Extracting out the Side for the IOC Trade from the CALL Instrument
        call_side = df_call[df_call['Type']=='T'].Side.iloc[0]
        
        # Slicing PUT Instrument IOC Trades based on the Delta
        df_put = put_ioc_df[put_ioc_df['ExchTstamp'].between(timestamp-delta, timestamp+delta)]
        
        # Filtering out the PUT IOC Trades based on the CALL Side, given for a Straddle - Both legs need to have the same side
        df_put = df_put[df_put['Side']==call_side]

        # Check for straddles
        if(len(df_put) != 0):
            if call_side=='B':
                # Finding out min qty of CALL and PUT Leg for Straddle qty traded
                minqty = min(df_call[df_call['Type']=='T'].qty.sum(), df_put.qty.sum())
                
                # Average of price and qty, and then multiplying it by minqty to get Straddle CALL traded value
                bought_value_call = (df_call.value.sum() / df_call[df_call['Type']=='T'].qty.sum()) * minqty
                # Average of price and qty, and then multiplying it by minqty to get Straddle PUT traded value
                bought_value_put = (df_put.value.sum() / df_put.qty.sum()) * minqty
                
                # Adding up all the values
                bought_straddles_value = bought_straddles_value + bought_value_call + bought_value_put
                bought_straddles_cnt += 1
            else:
                # Same calculation as above, just for Side = S
                minqty = min(df_call[df_call['Type']=='T'].qty.sum(), df_put.qty.sum())
                sold_value_call = (df_call.value.sum() / df_call[df_call['Type']=='T'].qty.sum()) * minqty
                sold_value_put = (df_put.value.sum() / df_put.qty.sum()) * minqty
                sold_straddles_value = sold_straddles_value + sold_value_call + sold_value_put
                sold_straddles_cnt += 1

    print('DELTA - {}'.format(deltaval))
    print('Bought Straddles Count : {}'.format(bought_straddles_cnt))
    print('Bought Straddles Value : {:,}'.format(round(bought_straddles_value,2)))
    print('Sold Straddles Count : {}'.format(sold_straddles_cnt))
    print('Sold Straddles Value : {:,}'.format(round(sold_straddles_value,2)))
    print('Net Straddles Traded Count : {}'.format(bought_straddles_cnt-sold_straddles_cnt))
    print('Net Straddles Traded Value : {:,}'.format(round(bought_straddles_value-sold_straddles_value,2)))
    print('------------------------------------------------')

In [6]:
delta_list = ['100 nanoseconds', '250 nanoseconds', '500 nanoseconds', '1 microsecond', '10 microseconds', '500 microseconds', '1 millisecond', '100 milliseconds', '1 second']

for deltaval in delta_list:
    find_straddles(deltaval)

DELTA - 100 nanoseconds
Bought Straddles Count : 0
Bought Straddles Value : 0
Sold Straddles Count : 0
Sold Straddles Value : 0
Net Straddles Traded Count : 0
Net Straddles Traded Value : 0
------------------------------------------------
DELTA - 250 nanoseconds
Bought Straddles Count : 0
Bought Straddles Value : 0
Sold Straddles Count : 0
Sold Straddles Value : 0
Net Straddles Traded Count : 0
Net Straddles Traded Value : 0
------------------------------------------------
DELTA - 500 nanoseconds
Bought Straddles Count : 0
Bought Straddles Value : 0
Sold Straddles Count : 0
Sold Straddles Value : 0
Net Straddles Traded Count : 0
Net Straddles Traded Value : 0
------------------------------------------------
DELTA - 1 microsecond
Bought Straddles Count : 0
Bought Straddles Value : 0
Sold Straddles Count : 0
Sold Straddles Value : 0
Net Straddles Traded Count : 0
Net Straddles Traded Value : 0
------------------------------------------------
DELTA - 10 microseconds
Bought Straddles Count